In [ ]:
!pip install git+https://github.com/EnergieID/entsoe-py.git
!pip install kafka-python


In [1]:
import csv
import logging
import os
import sys
import pandas as pd
from time import sleep
from typing import Dict
from urllib.parse import urlparse
import json 
from datetime import datetime


In [2]:
src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)

In [20]:
from api.entsoe import EntsoeDataDownloader
from config import Config

def main():
    year = 2024
    month = 7
    date_to_read = '2023-07-21'
    country_code = 'DE'
    data_type = 'load'  # Could be 'load', 'generation', etc.

    downloader = EntsoeDataDownloader(year, month, country_code)
    filename = downloader.download_data(data_type)
    return filename


In [21]:
filename = main()

ERROR:root:Failed to load logging config file: ./logs/logging.dev.ini doesn't exist


In [22]:
filename

'/tmp/2024_7_load.csv'

In [23]:
resource_path = '/tmp/2024_7_load.csv'

In [24]:
df = pd.read_csv(resource_path)

In [25]:
df.columns

Index(['date', 'load'], dtype='object')

In [43]:
_last_published_field_value = None

In [44]:
_field_name = 'date'

def is_record_processed(record,_last_published_field_value):
    if _last_published_field_value is None:
        return False
    record_value = datetime.fromisoformat(record[_field_name])
    return record_value <= _last_published_field_value
           
_index = 0
filters = {}
filter_funcs = {'datetome_to_publish': lambda row: row['date'][:10] == '2024-07-21'}
fields = ['date','load'] 
df = pd.read_csv(filename)
for _, row in df.iterrows():
    if all(filter_func(row) for filter_func in filter_funcs.values()):   
        record = {field: row[field] for field in fields}
        record['key_id'] = f'record_{_index}'
        _index += 1
        if is_record_processed(record,_last_published_field_value):
            continue
        _last_published_field_value = datetime.fromisoformat(record[_field_name])   
        print(datetime.fromisoformat(record['date']))

2024-07-21 00:00:00+02:00
2024-07-21 00:15:00+02:00
2024-07-21 00:30:00+02:00
2024-07-21 00:45:00+02:00
2024-07-21 01:00:00+02:00
2024-07-21 01:15:00+02:00
2024-07-21 01:30:00+02:00
2024-07-21 01:45:00+02:00
2024-07-21 02:00:00+02:00
2024-07-21 02:15:00+02:00
2024-07-21 02:30:00+02:00
2024-07-21 02:45:00+02:00
2024-07-21 03:00:00+02:00
2024-07-21 03:15:00+02:00
2024-07-21 03:30:00+02:00
2024-07-21 03:45:00+02:00
2024-07-21 04:00:00+02:00
2024-07-21 04:15:00+02:00
2024-07-21 04:30:00+02:00
2024-07-21 04:45:00+02:00
2024-07-21 05:00:00+02:00
2024-07-21 05:15:00+02:00
2024-07-21 05:30:00+02:00
2024-07-21 05:45:00+02:00
2024-07-21 06:00:00+02:00
2024-07-21 06:15:00+02:00
2024-07-21 06:30:00+02:00
2024-07-21 06:45:00+02:00
2024-07-21 07:00:00+02:00
2024-07-21 07:15:00+02:00
2024-07-21 07:30:00+02:00
2024-07-21 07:45:00+02:00
2024-07-21 08:00:00+02:00
2024-07-21 08:15:00+02:00
2024-07-21 08:30:00+02:00
2024-07-21 08:45:00+02:00
2024-07-21 09:00:00+02:00
2024-07-21 09:15:00+02:00
2024-07-21 0

In [45]:
_index

82

In [40]:
print(_last_published_field_value)

2024-07-21 20:30:00+02:00


In [41]:
import logging
from time import sleep
from config import Config
from api.entsoe import EntsoeDataDownloader

def main():
    
    data_downloader = EntsoeDataDownloader(
        year=int(Config.YEAR), 
        month=int(Config.MONTH), 
        country_code=Config.COUNTRY_CODE
    )
    filter_funcs = {'datetome_to_publish': lambda row: row['date'][:10] == Config.DATE_TO_READ}
    filename = data_downloader.download_data(data_type=Config.DATA_TYPE)
    df = pd.read_csv(filename)
    return df

In [42]:
df = main()

ERROR:root:Failed to load logging config file: ./logs/logging.dev.ini doesn't exist


In [15]:
df.head(5)

,date,load
0,2024-04-01 00:00:00+02:00,36256.0
1,2024-04-01 00:15:00+02:00,35809.0
2,2024-04-01 00:30:00+02:00,35612.0
3,2024-04-01 00:45:00+02:00,35275.0
4,2024-04-01 01:00:00+02:00,34757.0


In [6]:
Config.DATE_TO_READ

'2024-02-13'

In [7]:
Config.MONTH

'4'